In [ ]:
#CNN_LSTM
'''
 A cnn is made to work with two-dimensional image data. The CNN can be very effective at automatically extracting and 
 learning features from one-dimensional sequence data such as univariate time series data. A CNN model can be used in 
 a hybrid model with an LSTM backend where the CNN is used to interpret subsequences of input that together are 
 provided as a sequence to an LSTM model to interpret
'''

In [ ]:
#step1 The first step is to split the input sequences into subsequences that can be processed by the CNN model
#step2 we can first split our univariate time series data into input/output samples with four steps as input and one 
#as output
#step3 Each sample can then be split into two sub-samples, each with two time steps
#step4 The CNN can interpret each subsequence of two time steps and provide a time series of interpretations of the 
#subsequences to the LSTM model to process as input
#required structure: [samples, subsequences, timesteps, features]

In [ ]:
#We want to reuse the same CNN model when reading in each sub-sequence of data separately.
#This can be achieved by wrapping the entire CNN model in a TimeDistributed wrapper that
#will apply the entire model once per input


In [ ]:
'''
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, n_steps, n_features)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
'''

In [2]:
# univariate cnn lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [4]:

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
  X, y = list(), list()
  for i in range(len(sequence)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the sequence
    if end_ix > len(sequence)-1:
     break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 4
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
n_features = 1
n_seq = 2
n_steps = 2
X = X.reshape((X.shape[0], n_seq, n_steps, n_features))
# define model
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'),input_shape=(None, n_steps, n_features))) 
model.add(TimeDistributed(MaxPooling1D(pool_size=2))) 
model.add(TimeDistributed(Flatten())) 
model.add(LSTM(50, activation='relu')) 
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse') 

# fit model
model.fit(X, y, epochs=500, verbose=0)

                                                                                                           

In [6]:
x_input = array([60,70,80,90])
x_input = x_input.reshape((1, n_seq, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[101.41453]]
